In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.utils import degree
from torch_geometric.loader import DataLoader
from DrBC import DrBC
from tqdm import tqdm
import numpy as np


NUM_MIN = 200
NUM_MAX = 300
DIM = 256
LAYERS = 3
BATCH_SIZE = 16
Max_Eposide = 50
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", device)

Device: cuda


In [2]:
# Initialize the model
model = DrBC(dim=DIM, num_layers=LAYERS, aggregate_type='sum').to(device)

# Optimizer for the model
optimizer = optim.Adam(model.parameters())

loss = None

# Load the training graphs
graphs = torch.load(f"../Data/Synthetic_Train/{NUM_MIN}-{NUM_MAX}.pt", weights_only=False)
print("Loaded training graphs:", len(graphs))
# Create a DataLoader for the training graphs
loader = DataLoader(graphs, batch_size=BATCH_SIZE, shuffle=True)

for i in range(Max_Eposide):
    avg_loss = 0
    for batch in tqdm(loader):
        batch = batch.to(device)
        optimizer.zero_grad()

        # 計算圖的歸一化因子
        row, col = batch.edge_index
        deg = degree(row, batch.x.size(0), dtype=batch.x.dtype)
        deg_inv_sqrt = 1.0 / torch.sqrt(deg + 1)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        # 模型預測
        scores = model(batch, norm)

        # Real bc values
        real_bc = batch.y

        # 取得 batch 內各 graph 的資訊
        graph_ids, num_nodes_per_graph = torch.unique(batch.batch, return_counts=True)

        ranking_loss = 0.0

        for graph_id, num_nodes in zip(graph_ids, num_nodes_per_graph):
            mask = batch.batch == graph_id
            node_indices = torch.where(mask)[0]

            num_pairs = 5 * num_nodes.item()
            if num_nodes.item() < 2:
                continue  # 避免圖內節點不足 2 個導致錯誤

            idx = torch.randint(0, len(node_indices), (num_pairs, 2), device=device)
            u, v = node_indices[idx[:, 0]], node_indices[idx[:, 1]]

            s_u, s_v = scores[u], scores[v]
            b_ij = s_u - s_v
            real_bc_u, real_bc_v = real_bc[u], real_bc[v]
            y_ij = (real_bc_u > real_bc_v).float()  # 轉換成 0/1 標籤

            loss = F.binary_cross_entropy_with_logits(b_ij, y_ij)
            ranking_loss += loss  # 直接累加 loss，避免 .item() 影響計算圖

        ranking_loss /= BATCH_SIZE  # 取平均，確保 loss 的尺度穩定
        ranking_loss.backward()
        optimizer.step()
        avg_loss += ranking_loss.item()  # 這裡才使用 .item()

    avg_loss /= len(loader)
    print(f"Epoch {i+1}/{Max_Eposide}, Loss: {avg_loss:.4f}")       


# Save the model
torch.save({'model_state_dict': model.state_dict()}, "../Model/DrBC_model.pth")
print("Model saved successfully.")

Loaded training graphs: 1000


100%|██████████| 63/63 [00:03<00:00, 20.67it/s]


Epoch 1/50, Loss: 0.1597


100%|██████████| 63/63 [00:02<00:00, 25.95it/s]


Epoch 2/50, Loss: 0.1031


...

Epoch 48/50, Loss: 0.0685


100%|██████████| 63/63 [00:01<00:00, 34.73it/s]


Epoch 49/50, Loss: 0.0686


100%|██████████| 63/63 [00:01<00:00, 33.31it/s]

Epoch 50/50, Loss: 0.0692
Model saved successfully.


In [ ]:
import torch
import numpy as np
from scipy.stats import kendalltau
from torch_geometric.utils import degree
from torch_geometric.loader import DataLoader
from tqdm import tqdm

# Top-K 準確度與 Kendall's Tau 計算函數（針對 batch 內的每張圖）
def evaluate_graphs(y_pred, y_true, batch_indices, k_percentage=10):
    """
    針對 batch 內每張圖計算 Top-K 準確率與 Kendall's Tau，並取平均
    :param y_pred: 預測 BC 值 (Tensor)
    :param y_true: 真實 BC 值 (Tensor)
    :param batch_indices: batch 內圖的索引 (Tensor)
    :param k_percentage: 選擇 Top-K%
    :return: 平均 Top-K 準確率, 平均 Kendall's Tau
    """
    graph_ids = torch.unique(batch_indices)  # 取得 batch 內所有圖 ID
    top_k_acc_list = []
    tau_list = []
    
    for graph_id in graph_ids:
        mask = batch_indices == graph_id
        pred_vals = y_pred[mask].cpu().numpy().flatten()
        true_vals = y_true[mask].cpu().numpy().flatten()

        if len(pred_vals) < 2:
            continue  # 避免單節點圖影響計算
        
        # 計算 Top-K 準確率
        k = max(1, int(len(pred_vals) * k_percentage / 100))  # 確保至少選 1 個
        top_k_pred = np.argsort(-pred_vals)[:k]
        top_k_true = np.argsort(-true_vals)[:k]

        top_k_acc = len(set(top_k_pred) & set(top_k_true)) / k
        top_k_acc_list.append(top_k_acc)

        # 計算 Kendall's Tau-b，避免 NaN
        tau, _ = kendalltau(pred_vals, true_vals, variant="b")
        if not np.isnan(tau):
            tau_list.append(tau)

        # 計算完全排序的準確率
        sorted_pred = np.argsort(-pred_vals)
        sorted_true = np.argsort(-true_vals)
        correct_count = np.sum(sorted_pred == sorted_true)
        total_count = len(sorted_pred)
        accuracy = correct_count / total_count if total_count > 0 else 0
        # print(f"Graph ID: {graph_id.item()}, Accuracy: {accuracy:.4f}, Top-K Accuracy: {top_k_acc:.4f}, Kendall's Tau: {tau:.4f}")
    
    avg_top_k_acc = np.mean(top_k_acc_list) if top_k_acc_list else 0.0
    avg_tau = np.mean(tau_list) if tau_list else 0.0
    return avg_top_k_acc, avg_tau


# 讀取測試圖
test_graphs = torch.load(f"../Data/Synthetic_Validation/{10000}-{10000}.pt", weights_only=False)
print("Loaded test graphs:", len(test_graphs))

test_loader = DataLoader(test_graphs, batch_size=32, shuffle=False)

# 讀取模型
model = DrBC(dim=DIM, num_layers=LAYERS, aggregate_type='sum').to(device)
checkpoint = torch.load("../Model/DrBC_model.pth", weights_only=True)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()
print("Model loaded successfully.")

# 初始化計算變數
total_top_k_acc = 0
total_tau = 0
num_batches = 0
K = 1  # Top-K 百分比

with torch.no_grad():
    for batch in tqdm(test_loader):
        batch = batch.to(device)
        
        # 計算圖的歸一化因子
        row, col = batch.edge_index
        deg = degree(row, batch.x.size(0), dtype=batch.x.dtype)
        deg_inv_sqrt = 1.0 / torch.sqrt(deg + 1)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        # 預測節點的影響力分數
        pred_bc = model(batch, norm)

        # 針對 batch 內的每張圖分開計算 Top-K 準確率和 Kendall's Tau
        top_k_acc, tau = evaluate_graphs(pred_bc, batch.y, batch.batch, k_percentage=K)
        total_top_k_acc += top_k_acc
        total_tau += tau    
        num_batches += 1

# 計算最終結果
final_acc = total_top_k_acc / num_batches
final_tau = total_tau / num_batches
print(f"Final Top-{K}% Accuracy: {final_acc:.4f}, Final Kendall's Tau: {final_tau:.4f}")


Loaded test graphs: 30
Model loaded successfully.


100%|██████████| 1/1 [00:00<00:00,  3.07it/s]

Final Top-1% Accuracy: 0.9517, Final Kendall's Tau: 0.8314
